In [8]:
import os
import numpy as np
# The below two functions have been given by organizers in the data section in mask_functions.py
import tensorflow as tf
from tensorflow import keras
from datetime import datetime,timedelta
from matplotlib import pyplot as plt
import import_ipynb
from config import IMG_WIDTH,IMG_HEIGHT,N_CHANNELS,BATCH_SIZE,N_CLASS,NB_EPOCH 
from keras.losses import BinaryCrossentropy,binary_crossentropy
def mask2rle(img, width, height):
    '''
        Converts image of size width x height to RLE encoded form 
        Args:
            img : Image array
            width : width of image
            height : height of image
        Returns:
            encoded RLE String
    '''
    rle = []
    lastColor = 0;
    currentPixel = 0;
    runStart = -1;
    runLength = 0;

    for x in range(width):
        for y in range(height):
            currentColor = img[x][y]
            if currentColor != lastColor:
                if currentColor == 255:
                    runStart = currentPixel;
                    runLength = 1;
                else:
                    rle.append(str(runStart));
                    rle.append(str(runLength));
                    runStart = -1;
                    runLength = 0;
                    currentPixel = 0;
            elif runStart > -1:
                runLength += 1
            lastColor = currentColor;
            currentPixel+=1;

    return " ".join(rle)

def rle2mask(rle, width, height):
    '''
        Converts RLE encoded pixel to image of size width x height
        Example, rle '1 3 10 5' implies pixels 1,2,3 are to be included in the mask, as well as 14,15,16,17,18.
        Args:
            rle : encoded pixel in RLE form
            width : width of image
            height : height of image
        Returns:
            decoded image masks
    '''
    mask= np.zeros(width* height)
    array = np.asarray([int(x) for x in rle.split()])
    starts = array[0::2]
    lengths = array[1::2]

    current_position = 0
    for index, start in enumerate(starts):
        current_position += start
        mask[current_position:current_position+lengths[index]] = 255
        current_position += lengths[index]
    
    return mask.reshape(width, height)

def get_list_of_files(dir_name):
    '''
        For the given path, get the List of all files in the directory tree 
        INPUT:
            dir_name - path to the directory where files to be searched
        OUTPUT:
            list containing full path of all files present inside input directory
    '''
    # names in the given directory 
    list_of_files = os.listdir(dir_name)
    all_files = []
    for entry in list_of_files:
        full_path = os.path.join(dir_name,entry)
        if os.path.isdir(full_path):
            all_files = all_files + get_list_of_files(full_path)
        else:
            all_files.append(full_path)
    return all_files


def dice_coef(y_true, y_pred,smooth = 1e-5):
    '''
        Given true and predicted pixel values it calculates dice_coefficient
        Args:
            y_true : true pixel values
            y_pred : predicted pixel values by model
            smooth : small value to avoid divide by zero error
        Returns:
            dice_coefficient value
    '''
    y_true_f = tf.keras.layers.Flatten()(y_true)
    y_pred_f = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)


def combined_loss(y_true, y_pred):
    '''
        Given true and predicted pixel values it calculates sum of dice loss and binary corss entrpy loss
        Args:
            y_true : true pixel values
            y_pred : predicted pixel values by model
            smooth : small value to avoid divide by zero error
        Returns:
            sum of BCE loss + Dice loss
    '''
    return 1.0 - dice_coef(y_true, y_pred) + binary_crossentropy(y_true, y_pred)

# define custom learning rate schedule
class CosineAnnealingLearningRateSchedule(tf.keras.callbacks.Callback):
    # constructor
    def __init__(self, n_epochs, n_cycles, lrate_max, verbose=0):
        self.epochs = n_epochs
        self.cycles = n_cycles
        self.lr_max = lrate_max
        self.lrates = list()
 
    # calculate learning rate for an epoch
    def cosine_annealing(self, epoch, n_epochs, n_cycles, lrate_max):
        epochs_per_cycle = np.floor(n_epochs/n_cycles)
        cos_inner = (np.pi * (epoch % epochs_per_cycle)) / (epochs_per_cycle)
        return lrate_max/2 * (np.cos(cos_inner) + 1)
 
    # calculate and set learning rate at the start of the epoch
    def on_epoch_begin(self, epoch, logs=None):
        # calculate learning rate
        lr = self.cosine_annealing(epoch, self.epochs, self.cycles, self.lr_max)
        # set learning rate
        K.set_value(self.model.optimizer.lr, lr)
        # log value
        self.lrates.append(lr)
        
def set_log_checkpoint(name):
    '''
        helper function to set model checkpoint, tensorboard callback
        Args:
           name: the name of the model
        Returns:
                model checkpoint callback , tensorboard callback, learningrate schedule callback
            
    '''
    global CHECKPOINT_PATH
    CHECKPOINT_PATH = "training/" + name + ".ckpt"
    checkpoint_dir = os.path.dirname(CHECKPOINT_PATH)
    # Create a callback that saves the model's weights
    cp_callback = keras.callbacks.ModelCheckpoint(filepath=CHECKPOINT_PATH,
                                                    save_weights_only=True,
                                                    verbose=1,
                                                    monitor='val_loss',
                                                    save_best_only=True)
    # Load the TensorBoard notebook extension
    %load_ext tensorboard
    # Clear any logs from previous runs
    ! rm -rf ./logs/ 
    # Set up log directory
    logdir = os.path.join("logs", datetime.now().strftime("%Y%m%d-%H%M%S"))
    #print(logdir)
    %tensorboard --logdir $logdir
    tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=1)
    ca = CosineAnnealingLearningRateSchedule(NB_EPOCH, 10, 0.01)
    # early = keras.callbacks.EarlyStopping(monitor='val_recall', min_delta=0, patience=40, verbose=1, mode='auto')
    return cp_callback,tensorboard_callback,ca

def plot_metrics(history):
    '''
        Helper function to plot traing history such as train and validation loss and dice coeffcient 
        Args:
           history: Model training history obejct 
        Returns:
                None
            
    '''
    # list all data in history
    print(history.history.keys())
    # summarize history for DiceCoef
    fig, axes = plt.subplots(1, 2,figsize=(20,7))
    axes[0].plot(history.history['dice_coef'])
    axes[0].plot(history.history['val_dice_coef'],linestyle="--")
    axes[0].set_title('Model epoch vs DiceCoef', fontsize=18, pad=15)
    axes[0].set_ylabel('DiceCoef',size=14)
    axes[0].set_xlabel('epoch',size=14)
    plt.legend(['train', 'test'], loc='upper left')
    #plt.show()
    # summarize history for loss
    #plt.subplot(122)
    axes[1].plot(history.history['loss'])
    axes[1].plot(history.history['val_loss'],linestyle="--")
    axes[1].set_title('model epoch vs loss', fontsize=18, pad=15)
    axes[1].set_ylabel('loss',size=14)
    axes[1].set_xlabel('epoch',size=14)
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()